# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

pd.set_option('max_rows', 4, 'max_columns', 10, 'max_colwidth', 200, 'precision', 2)
# ?fetch_20newsgroups

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**
 
<span style="color:black">You will construct a term frequency-inverse document frequency (TF-IDF) document term matrix (DTM) using a subset of the data in the [20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html), which is accessible using the Scikit-Learn library. This dataset consists of posts from newsgroups for twenty different categories. Begin by observing the available categories in this dataset.

In [ ]:
LsAllCats = fetch_20newsgroups()['target_names']
print(LsAllCats)

### **Retrieve Two Categories**

<span style="color:black"> You will only use two of these categories: <em>rec.autos</em> (autos, a subcategory of recreational) and <em>sci.space</em> (space, a subcategory of science) to construct a DataFrame of observations and labels. Retrieve this data and notice that each entry has a label `Y`, 0 for auto and 1 for space.
    
`dfTxtY` is used for data display only. Otherwise, it is not needed in this notebook. 

In [ ]:
LsCats = ['rec.autos', 'sci.space']
# download two newsgroups as a tuple of string (message text) and a number (outcome 0 or 1)
TsnTxtY = fetch_20newsgroups(subset='all', categories=LsCats, shuffle=True, random_state=0, return_X_y=True)

dfTxtY = pd.DataFrame(TsnTxtY, index=['Txt','Y']).T
dfTxtY['cat'] = np.array(LsCats)[dfTxtY.Y.tolist()] # retrieve a string category for each numeric label
dfTxtY

### **Create Training Inputs and Outputs**

<span style="color:black"> You can now use the `train_test_split()` function to split the dataset into training inputs with corresponding outputs and validation inputs with corresponding outputs.

In [ ]:
tTxt, vTxt, tY, vY = train_test_split(TsnTxtY[0], TsnTxtY[1], test_size=0.25, random_state=0)

print('tTxt:', len(tTxt), '\t tY:', len(tY))
print('vTxt:', len(vTxt), '\t vY:', len(vY))

<span style="color:black">To get a sense of the text used in the posts, print the top few training posts with their labels.

In [ ]:
print(tY[:2])
tTxt[:2]

### **Fitting and Transforming**

<span style="color:black"> You will now build a TF-IDF DTM from the training data. Begin by initializing the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) object with English stopwords, and then fit and transform this object on the <em>training</em> text, `tTxt`. Notice that the vectorizer does not use the validation observations, because the model should have no knowledge of the validation observations, which are used to evaluate the model.
    
<span style="color:black"> <b>Note:</b> Check the documentation to view all default parameters, noting that lowercasing is set to `True` by default.

In [ ]:
tv = TfidfVectorizer(stop_words='english', min_df=2)
smtX = tv.fit_transform(tTxt)  # sparse matrix format
smtX

### **Transform to Numeric Vectors**

<span style="color:black">Now you will use the trained vectorizer to transform each of the <em>validation</em> posts into a numeric vector.

<span style="color:black"><b>Note:</b> No training is done here. You are using the previously learned vocabulary only.

In [ ]:
smvX = tv.transform(vTxt)  # re-use vocabulary from train corpus, do not build a new vocabulary
smvX

### **Visualize Values in the Sparse Matrix**

<span style="color:black"> Observe several values from the TF-IDF DTM, which is in a sparse matrix format.

In [ ]:
print(smtX[:30,:30])  # only non-zero locations and values are stored

<span style="color:black">Notice that the sparse matrix format is a container with elements in the form `(i,j) v`, where `(i,j)` is the row/column index of the non-zero value `v` in the underlying matrix. This is the so-called **adjacency* format, which ignores zero elements to save storage space. This is an efficient storage format, if the underlying matrix is mostly zero values. 

<span style="color:black">Compute the fraction of zeros in this TF-IDF matrix.</span>

In [ ]:
nCountWeights = smtX.shape[0] * smtX.shape[1]
print(f'fraction of zeros in sparse matrix: {1 - smtX.count_nonzero() / nCountWeights:.4f}')

<span style="color:black">Notice that 99.25% are zeros, which indicates that the matrix is highly sparse with only a few weights distributed around the sentences. </span>

### **Designing Models on Sparse Observations**

<span style="color:black">Designing a mathematical model, such as logistic regression, on sparse observations can be wasteful because each feature may need a parameter that contributes to the final prediction. Here, most features are zeros, and so, its corresponding parameters in the model will not make meaningful contribution. Yet, estimation of each parameter requires computational resources and is also a source of uncertainty. Modern vectorization methods allow compressing the full sentence or a document into a 500 dimensional dense vector (with no zeros), which is far more efficient. You will see these models later.

<span style="color:black">Display a few rows and columns of the training dataframe to show that the matrix is highly sparse. 

In [ ]:
plt.rcParams['figure.figsize'] = [20, 4]   # plot wider figures
LsIdx = [f'{len(s.split())}, {s[:30]}...' for s in tTxt] # number of words and sentence excerpt

dftX = pd.DataFrame(smtX.toarray(), index=LsIdx, columns=tv.get_feature_names())

ax = sns.heatmap(dftX.iloc[:10,5000:5050], annot=True, cmap='coolwarm', cbar=False);
tmp = ax.set_title('Data Matrix: TF-IDF Matrix of training features');
print(f'DTM shape, observations x features: {dftX.shape}')

<span style="color:black">Now display a few rows and columns from the validation DataFrame.
    
<span style="color:black"><b>Note:</b> The number of features (i.e. columns) and their order must match the training matrix.

In [ ]:
plt.rcParams['figure.figsize'] = [20, 4]   # plot wider figures
LsIdx = [f'{len(s.split())}, {s[:30]}...' for s in vTxt] # number of words and sentence excerpt

dfvX = pd.DataFrame(smvX.toarray(), index=LsIdx, columns=tv.get_feature_names())

ax = sns.heatmap(dfvX.iloc[:10,5000:5050], annot=True, cmap='coolwarm', cbar=False);
tmp = ax.set_title('Data Matrix: TF-IDF Matrix of training features');
print(f'DTM shape, observations x features: {dfvX.shape}')

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now, equipped with these concepts and tools, you will practice a few related tasks.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

### **Overview**

As you may have guessed, it should be easier to classify poorly related topics than those that are very similar. For example, classifying autos versus motorcycles should be harder than autos versus medicine. You will verify this hypothesis. Short of creating a machine learning model (which you will do later), you will measure the overlap in vocabulary between `rec.autos` and each other category. Since each news category can vary in size, you would need to standardize the similarity metric. 

[**Jaccard Similarity**](https://www.statology.org/jaccard-similarity/) is a suitable simple choice (we covered it earlier, but will see it again in future courses). It is defined as the size of the intersection between two sets divided by the size of their union. So, identical sets will have similarity of one and completely disjoint sets will have a similarity of zero, because there is no intersection.


## Task 1

For each news category name in `LsAllCats` retrieve the count of all news posts. Order the results by decreasing count.

<b>Hint:</b> You will need a list of all output labels, which you can retrieve with <code>fetch_20newsgroups(...,return_X_y=True)[1]</code>. You can then group and count each category ID by either using the Pandas dataframe or the <code>collections.Counter</code> object. You must also retrieve the string category name using a numeric category ID.


In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
from collections import Counter
aY = fetch_20newsgroups(subset='all', return_X_y=True)[1]  # all news category IDs
DnCounts = Counter(aY) # dictionary of news category IDs and their counts
sorted([(v, LsAllCats[k]) for k, v in DnCounts.items()], reverse=True)
# %time [len(fetch_20newsgroups(subset='all', categories=[c])['data']) for c in LsAllCats] # slower alternative
</pre>
</details> 
</font>
<hr>

## Task 2

Build a `JaccardSim(A=set(),B=set())` function for Jaccard Similarity for the sets $A$ and $B$. This function returns a number between 0 and 1, where 1 indicates high similarity between sets and 0 indicates no similarity. 

The formula is $|A\cap B|/|A\cup B|$, where $|A|$ is the size or cardinality of the set $A$, i.e. number of elements. 

Test your function with different sets to ensure it is working correctly.

<b>Hint:</b> Use <code>intersection</code> and <code>union</code> methods of the set object. See documentation.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
JaccardSim = lambda A=set(),B=set(): len(A.intersection(B)) / len(A.union(B))
JaccardSim({1,2,3}, {1,2,3})
JaccardSim({1,2,3}, {2,3,4})
JaccardSim({1,2,3}, {'abs'})
JaccardSim({1,2,3}, {})
</pre>
</details> 
</font>
<hr>

## Task 3

Build a function `GetVocabSet(sCat='sci.med')` which takes a category name and returns a set of vocabulary words, which results from fitting a `TfidfVectorizer()` (with its default parameters) on the posts in category `sCat`. 

Thus, `len(GetVocabSet())` should return a set of 21599 words.

<b>Hint:</b> Don't forget to set <code>subset='all'</code> in <code>fetch_20newsgroups</code>. See its documentation. You can return the full data bunch object and then retrieve its <code>'data'</code> attribute containing news posts. Once you fit a vectorizer, use the <code>vocabulary_</code> attribute. See documentation.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
tv = TfidfVectorizer()
GetVocabSet = lambda sCat='sci.med': set(tv.fit(fetch_20newsgroups(subset='all', categories=[sCat])['data']).vocabulary_.keys())
len(GetVocabSet())
</pre>
</details> 
</font>
<hr>

## Task 4

For each category name in `LsAllCats`, use `GetVocabSet()` to retrieve the vocabulary set and compute its Jaccard Similarity with the vocabulary set from the `rec.autos` category. The resulting list of tuples (similarity score, category) should be ordered by decreasing similarity.

<b>Hint:</b> Here you can build a couple of loops. One to retrieve vocabulary and another to compute similarities. You can also do it all in a single loop.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
DsVoc = {s:GetVocabSet(s) for s in LsAllCats}
sorted([(JaccardSim(DsVoc['rec.autos'], v), k)  for k, v in DsVoc.items()], reverse=True)
</pre>
</details> 
</font>
<hr>